<a href="https://colab.research.google.com/github/rklepov/hse-cs-ml-2018-2019/blob/homework/08-spark/HW/hw1/%D0%B4%D0%B71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip search pyspark | grep 2.4.0 | grep INSTALLED || pip install pyspark==2.4.0 findspark

  INSTALLED: 2.4.0


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/"

import findspark
findspark.init('/usr/local/lib/python3.6/dist-packages/pyspark/')

import pyspark

## NB: python3-only

In [3]:
from zipfile import ZipFile
from io import BytesIO
import urllib.request

ZipFile.extract(
    ZipFile(
        BytesIO(
            urllib
            .request
            .urlopen('http://s3.amazonaws.com/alexa-static/top-1m.csv.zip')
            .read()
        )
    ),
    'top-1m.csv'
)

'/content/top-1m.csv'

In [4]:
!head -5 /content/top-1m.csv

1,google.com
2,youtube.com
3,facebook.com
4,baidu.com
5,wikipedia.org


# Дз №1. Мапредьюс руками

Напишите маппер и редьюсер для задачи обратного индекса по рейтингу сайтов алексы.
Задача построения обратного индекса означает, что нужно разбить все домены в рейтинге на части(токены) по символу точки, а на выходе получить пары (токен, все сайты в названии которых содержался токен). Пример:

```
(
    google.com,
    google.ru,
    ya.ru,
    hse.ru
)

```
должно на выходе превратиться в:

```python
(
    (com, (google.com,)),
    (google, (google.com, google.ru)),
    (ru, (google.ru, ya.ru, hse.ru)),
    (ya, (ya.ru, )),
    (hse, (hse.ru, ))
)
```


In [0]:
with open('/content/top-1m.csv') as alexa:
    alexa_domains = [s.strip().split(",")[1] for s in alexa]

## 1.1 Маппер

In [0]:
def mapper(domain):
    """
    Маппер для обратного индекса. Разбивает доменное имя на токены и помечает каждый токен доменом. 
    Пример: google.com -> ('com', 'google.com'), ('google', 'google.com')
    """
    # Ваш код здесь
    
    return ((token, domain) for token in domain.split('.')[::-1])

In [7]:
list(mapper('google.com'))

[('com', 'google.com'), ('google', 'google.com')]

## 1.2 Редьюсер

In [0]:
def reducer(token_domain_pairs):
    """
    Редьюсер для обратного индекса. По набору кортежей, созданных маппером, возвращает кортеж 
    вида (токен, (домен1, домен2, ...)
    Пример:
    (('google', 'google.com'), ('google', 'google.se')) -> ('google', ('google.com', 'google.se'))
    Примечание: гарантируется, что у всех кортежей на входе оданковый токен.
    """
    # Ваш код здесь

    return ( token_domain_pairs[0][0], [pair[1] for pair in token_domain_pairs] )

In [9]:
list(reducer((('google', 'google.com'), ('google', 'google.se'))))

['google', ['google.com', 'google.se']]

## bonus:

Напишите также несколько тестов, чтобы проверить корректность маппера и редьюсера

например таких:

In [0]:
assert set(mapper("ya.ru")) == {("ya", "ya.ru"), ("ru", "ya.ru")}
assert set(mapper("com")) == {("com", "com")}

def reducer_test(testcase, expected_result):
    actual_result = reducer(testcase)
    assert actual_result[0] == expected_result[0]
    assert set(actual_result[1]) == set(expected_result[1])
    
reducer_test(
    (('google', 'google.com'), ('google', 'google.se')),
    ('google', ('google.com', 'google.se'))
)

In [0]:
def flatmap(mapper_func, iterable):
    from itertools import chain
    return tuple(chain.from_iterable(map(mapper_func, iterable)))

#### проверим как работает ваш маппер на датасете:

In [12]:
flatmap(mapper, alexa_domains[:10])

(('com', 'google.com'),
 ('google', 'google.com'),
 ('com', 'youtube.com'),
 ('youtube', 'youtube.com'),
 ('com', 'facebook.com'),
 ('facebook', 'facebook.com'),
 ('com', 'baidu.com'),
 ('baidu', 'baidu.com'),
 ('org', 'wikipedia.org'),
 ('wikipedia', 'wikipedia.org'),
 ('com', 'qq.com'),
 ('qq', 'qq.com'),
 ('com', 'taobao.com'),
 ('taobao', 'taobao.com'),
 ('com', 'amazon.com'),
 ('amazon', 'amazon.com'),
 ('com', 'tmall.com'),
 ('tmall', 'tmall.com'),
 ('com', 'vk.com'),
 ('vk', 'vk.com'))

## 1.3 Шаффл
напишите функцию, которая сгруппирует всю выдачу всех мапперов по токену, так чтобы соблюдалось предположение, принятое нами в определении редьюсера:

> `Примечание: гарантируется, что у всех кортежей на входе оданковый токен.`

#### Примечание: в идеале, в реализации шафла не пользоваться питоньими диктами (`{k:v}` и т.д )

In [0]:
def shuffle(dataset):
    """
    Группирует с выводом мапперов по ключу. Пример:
    
    (
        ('google', 'google.com'),
        ('com', 'google.com'),
        ('youtube', 'youtube.com'),
        ('com', 'youtube.com')
    )

    --> shuffle все превращает в --> 

    (
        [('com', 'google.com'), ('com', 'youtube.com')],
        [('google', 'google.com')],
        [('youtube', 'youtube.com')],
    )

    :param dataset:
    :return:
    
    """
    # Ваш код здесь
    
    import itertools as it
    keyf = lambda x: x[0]
    grouped = it.groupby(sorted(dataset, key=keyf), key=keyf)
    return ( list(grp) for key, grp in grouped )

### Собираем все вместе

In [0]:
mapper_output = flatmap(mapper, alexa_domains)
shuffled = shuffle(mapper_output)
reduced = map(reducer, shuffled)

reversed_idx = dict(reduced)

Должно посчитаться секунд за пять. Если не получается и считается долго, то прежде чем оптимизировать код попробуйте подставить небольшое подмножество `alexa_domains`, например `alexa_domains[:100]`. Корректность кода важнее быстродействия.

`reversed_idx` должен содержать то что нужно для поиска по токену:

In [15]:
reversed_idx['yandex']

['yandex.ru',
 'yandex.kz',
 'yandex.by',
 'yandex.com',
 'yandex.com.tr',
 'yandex.net',
 'yandex.ua',
 'yandex.uz',
 'yandex.fr',
 'eda.yandex',
 'yandex.md',
 'driver.yandex',
 'clickhouse.yandex',
 'chef.yandex',
 'yandex.com.ge',
 'yandex.org.kz',
 'yandex.uk.com',
 'yandex.tm',
 'yandex.gb.net',
 'auto.yandex',
 'yandex.az',
 'yandex.lv',
 'yandex.im',
 'yandex.co.il',
 'maps.yandex',
 'yandex.kg',
 'garage.yandex',
 'yandex.tj']

In [16]:
reversed_idx['wikipedia']

['wikipedia.org',
 'wikipedia.de',
 'wikipedia.nym.by',
 'wikipedia.it',
 'wikipedia.nom.al',
 'wikipedia.no',
 'wikipedia.at',
 'wikipedia.fr',
 'wikipedia.nom.si',
 'wikipedia.ma',
 'wikipedia.nom.ae',
 'wikipedia.blogfa.com']

### *References*

[1] https://docs.python.org/3.8/library/itertools.html#itertools.groupby

[2] https://realpython.com/python-itertools/#building-relay-teams-from-swimmer-data

[3] https://medium.com/@jasonrigden/a-guide-to-python-itertools-82e5a306cdf8